In a causal model we generate data in two sequential steps:
- First pick the hidden states from their prior distribution. 
- Then pick the visible states from their conditional distribution given the hidden states. 

$$ p(v) = \sum_h p(h)p(v|h) $$

<img src='pic\hinton causal model generate data.png'>

Boltzmann Machine is **not** a causal generative model. Instead, eveything is defined in terms of the energies of joint configurations of the visible and hidden units. 


$$\large 总公式: p(v,h) \propto e^{-E(v,h)} $$

$p(v,h)$. finding the network in that joint configuration after we have updated all of the stochastic binary units many times

$$ -E(v,h) = \underbrace{\sum_{i\in vis} v_ib_i}_{visible层的基本势能} + \overbrace{\sum_{k\in hid} h_kb_k}^{hidden层的基本势能} + \underbrace{\sum_{i<j} v_iv_jw_{ij}}_{visual层之间的连接} + \overbrace{\sum_{i,k} v_ih_kw_ik}^{visual层和hidden层之间的连接} + \underbrace{\sum_{k<l} h_kh_lw_{kl}}_{hidden层之间的连接}   $$


$ -E(v,h) $: 

Energy with configuration, **v** on the visible units and **h** on the hidden units. 

$ \sum_{i\in vis} v_ib_i + \sum_{k\in hid} h_kb_k$: 

$v_i$ is the binary state of unit i in v.  $b_k$ is the bias of unit k. bias 决定了该unit的一个基本势能。

$ \sum_{i<j} v_iv_jw_{ij} $

$ i < j $ ensure indexes every non-identical pair of i and j once. 



$ p(v,h) = \cfrac{exp(-E(v,h))}{\sum_{u,g} exp(-E(u,g))} $  

u, g 是v , h 的全集表现
当我我们确定了联合概率分布的时候，我们就可以通过积分消解不想要的量 

$ p(v) = \cfrac{\sum_h exp(-E(v,h))}{\sum_{u,g} exp(-E(u,g))} $ 

$ p(h) = \cfrac{\sum_v exp(-E(v,h))}{\sum_{u,g} exp(-E(u,g))} $ 

$ p(v|h) = \cfrac{exp(-E(v,h))}{\sum_u exp(-E(u,g))} $ 

$ p(h|v) = \cfrac{exp(-E(v,h))}{\sum_g exp(-E(u,g))} $ 



<img src='pic\hinton example of how weights define a distribution.png'>

这里摘录了网上对该问题的理解：http://blog.csdn.net/mytestmy/article/details/9150213

**求解的目标——让RBM网络的表示Gibbs分布最大可能的拟合输入数据。也可以认为是让RBM网络表示的Gibbs分布与输入样本的分布尽可能地接近。**

假设Ω表示样本空间，q是输入样本的分布，即q(x)表示训练样本x的概率， q其实就是要拟合的那个样本表示分布的概率；再假设p是RBM网络表示的Gibbs分布的边缘分布（只跟可视节点有关，隐藏节点是通过积分去掉了，可以理解为可视节点的各个状态的分布），输入样本的集合是S，那现在就可以定义样本表示的分布和RBM网络表示的边缘分布的KL距离

$ KL(q\mid\mid p) = \sum_{x\in\Omega} q(x) \ln\cfrac{q(x)}{p(x)} = \sum_{x\in\Omega} q(x)\ln q(x) - \sum_{x \in \Omega} q(x) \ln p(x) $

如果输入样本表示的分布与RBM表示的Gibbs分布完全符合，这个KL距离就是0，否则就是一个大于0的数。

第一项其实就是输入样本的熵（熵的定义），输入样本定了熵就定了；第二项没法直接求，但是如果用蒙特卡罗抽样，让抽中的样本是输入样本（输入样本肯定符合分布q(x)），第二项可以用$\frac{1}{l} \sum_{x \in \Omega} \ln p(x)$来估计，其中的l表示训练样本个数。由于KL的值肯定是不小于0，所以第一项肯定不小于第二项，让第二项取得最大值，就能让KL距离最小；最后，还可以发现，最大化$\frac{1}{l} \sum_{x \in \Omega} \ln p(x) $，相当于最大化 $\sum_{x \in \Omega} \ln p(x)，而这就是极大似然估计。

结论就是求解输入样本的极大似然，就能让RBM网络表示的Gibbs分布和样本本身表示的分布最接近。

这就是为什么RBM问题最终可以转化为极大似然来求解。
既然要用极大似然来求解，这个当然是有意义的——当RBM网络训练完成后，如果让这个RBM网络随机发生若干次状态（当然一个状态是由(v,h)组成的），这若干次状态中，可视节点部分（就是v）出现训练样本的概率要最大。

这样就保证了，在反编码（从隐藏节点到可视节点的编码过程）过程中，能使训练样本出现的概率最大，也就是使得反编码的误差尽最大的可能最小。

例如一个样本(1,0,1,0,1)编码到(0,1,1)，那么，(0,1,1)从隐藏节点反编码到可视节点的时候也要大概率地编码到(1,0,1,0,1)。

Now we back to hinton's notes

$ \cfrac{d \log p(v)}{d w_{ij}} = \langle s_i s_j \rangle_v- \langle s_i s_j \rangle_{model} $

当对log probability of one training vector求导时，其公式是：
把一条v记录clamp on the visible units 后达到的热平衡状态 减去 没有任何v记录clamping 是的热平衡状态

$ \Delta w_{ij} \propto \langle s_i s_j \rangle_{data} - \langle s_i s_j \rangle_{model} $

** Postive phase **

- Clamp a data vector on the visible units and set the hidden units to random binary states

- Update the hidden units one a a time until the network reaches thermal equilibrium at a temperature of 1. 

- Sample $<s_i s_j>$ for **every** connected pair of units

- Repeat for all data vectors in the training set and average

** Negative phase **

- Set **all** the units to random binary states (means no data vector clamping on visible units)

- Update all the units one at a time until the network reaches termal equilibrium at a temperature of 1. 

- Sample $<s_i s_j>$ for every connected pair o funits. 

- Repeat many times (how many?) and average to get good estimates. 


### Restricted Boltzmann Machines

We restrict the connectivity to make inference and learning easier. 

- Only one layer of hidden units. 

- No connections between hidden units. 

- No connection between visible units. 

<img src='pic\RBM network by hinton.png'>

In an RBM it only takes one step to reach termal equilibrium when the visible units are clamped. So that we can quickly get the exact value of : $ <v_i h_j>_v$

probably of $h_j$ is turn on: indepedent with other hidden units. 

$ p(h_j = 1) = \cfrac {1}{1+ exp(-(b_j + \sum_{i \in vis} v_iw_{ij} ))}$

** PCD: An efficient mini-batch learning procedure for RBM **

** Postive phase **

- Clamp a datavector on the visible units. 

- Compute the exact value of $<v_i h_j>$ for all pairs of a visible and a hidden unit. 

- For every connected pair of units, average $<v_i h_j>$ over all data in the mini-batche. 

** Negative phase **

- Keep a set of "fantasy particles". Each particle has a value that is a global configuration. 

- Update each fantasy particle a few times using alternating parallel updates. 

- For every connected pair of units, average $v_ih_j$ over all the fantasy particles. 



<img src='pic\an inefficient version of the Boltzmann machine learning algorithm for an RBM.png'>

$<v_ih_j>^0$ means $<v_ih_j>$ at time 0

Start with a training vector on the visible units. Then alternate between updating all the hidden units in parallel and updating all the visible units in parallel. 

由于hidden units相互独立，visible unit发起的更新可以对所有的hidden unit 并行计算的。当进行了无限多次的时候，达到热平衡状态，这时，我们称其配置为"fantasy".

$ \Delta w_{ij} = \epsilon (<v_ih_j>^0 - <v_ih_j>^\infty) $

#### 神奇的CD1

<img src='pic\CD1.png'>

$ \Delta w_{ij} = \epsilon (<v_ih_j>^0 - <v_ih_j>^1) $